In [1]:
import pandas as pd
import json

In [2]:
df= pd.read_csv('/Users/juuuu/Desktop/train.csv', nrows=5000)
df.head()

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA,POLYLINE
0,1372636858620000589,C,NaN,NaN,20000589,1372636858,A,False,"[[-8.618643,41.141412],[-8.618499,41.141376],[..."
1,1372637303620000596,B,NaN,7.0,20000596,1372637303,A,False,"[[-8.639847,41.159826],[-8.640351,41.159871],[..."
2,1372636951620000320,C,NaN,NaN,20000320,1372636951,A,False,"[[-8.612964,41.140359],[-8.613378,41.14035],[-..."
3,1372636854620000520,C,NaN,NaN,20000520,1372636854,A,False,"[[-8.574678,41.151951],[-8.574705,41.151942],[..."
4,1372637091620000337,C,NaN,NaN,20000337,1372637091,A,False,"[[-8.645994,41.18049],[-8.645949,41.180517],[-..."


In [3]:
df=df.drop_duplicates().reset_index(drop=True)

In [4]:
#df['MISSING_DATA'].value_counts()

In [5]:
#dropping missing data
df.drop(df[df["MISSING_DATA"]==True].index, axis=0, inplace=True)

In [6]:
# Convert the polyline string to a list
df['POLYLINE'] = df['POLYLINE'].apply(json.loads)

In [7]:
null_count = df.isnull().sum()

print(null_count)

TRIP_ID            0
CALL_TYPE          0
ORIGIN_CALL     3820
ORIGIN_STAND    2516
TAXI_ID            0
TIMESTAMP          0
DAY_TYPE           0
MISSING_DATA       0
POLYLINE           0
dtype: int64


In [8]:
# Get the start and end points
df['start_point'] = df['POLYLINE'].apply(lambda x: x[0] if x else 'NA')
df['end_point'] = df['POLYLINE'].apply(lambda x: x[-1] if x else 'NA')


In [9]:
# Extract longitude and latitude from start_point and end_point
df['start_point_longitude'] = df['start_point'].apply(lambda x: x[0] if x else 'NA')
df['start_point_latitude'] = df['start_point'].apply(lambda x: x[1] if x else 'NA')
df['end_point_longitude'] = df['end_point'].apply(lambda x: x[0] if x else 'NA')
df['end_point_latitude'] = df['end_point'].apply(lambda x: x[1] if x else 'NA')


In [10]:
#Extract pick up time
df['pickup_datetime'] = pd.to_datetime(df['TIMESTAMP'], unit='s')

In [11]:
columns_to_drop = ['MISSING_DATA', 'POLYLINE','start_point','end_point','TIMESTAMP','ORIGIN_STAND','ORIGIN_CALL','TRIP_ID']
df.drop(columns=columns_to_drop, inplace=True)

In [12]:
df.head()

,CALL_TYPE,TAXI_ID,DAY_TYPE,start_point_longitude,start_point_latitude,end_point_longitude,end_point_latitude,pickup_datetime
0,C,20000589,A,-8.618643,41.141412,-8.630838,41.154489,2013-07-01 00:00:58
1,B,20000596,A,-8.639847,41.159826,-8.66574,41.170671,2013-07-01 00:08:23
2,C,20000320,A,-8.612964,41.140359,-8.61597,41.14053,2013-07-01 00:02:31
3,C,20000520,A,-8.574678,41.151951,-8.607996,41.142915,2013-07-01 00:00:54
4,C,20000337,A,-8.645994,41.18049,-8.687268,41.178087,2013-07-01 00:04:51


In [13]:
df['trip_id']=df.index

In [14]:
datetime_dim = df[['pickup_datetime']]
datetime_dim['pick_hour'] = datetime_dim['pickup_datetime'].dt.hour
datetime_dim['pick_day'] = datetime_dim['pickup_datetime'].dt.day
datetime_dim['pick_month'] = datetime_dim['pickup_datetime'].dt.month
datetime_dim['pick_year'] = datetime_dim['pickup_datetime'].dt.year
datetime_dim['pick_weekday'] = datetime_dim['pickup_datetime'].dt.weekday
datetime_dim['datetime_id'] = datetime_dim.index

/var/folders/hy/4bfwf9k51gv94pny320n7kqc0000gn/T/ipykernel_85427/777856911.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datetime_dim['pick_hour'] = datetime_dim['pickup_datetime'].dt.hour
/var/folders/hy/4bfwf9k51gv94pny320n7kqc0000gn/T/ipykernel_85427/777856911.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datetime_dim['pick_day'] = datetime_dim['pickup_datetime'].dt.day
/var/folders/hy/4bfwf9k51gv94pny320n7kqc0000gn/T/ipykernel_85427/777856911.py:4: SettingWithCopyWarning: 
A value is trying 

In [15]:
datetime_dim.head()

,pickup_datetime,pick_hour,pick_day,pick_month,pick_year,pick_weekday,datetime_id
0,2013-07-01 00:00:58,0,1,7,2013,0,0
1,2013-07-01 00:08:23,0,1,7,2013,0,1
2,2013-07-01 00:02:31,0,1,7,2013,0,2
3,2013-07-01 00:00:54,0,1,7,2013,0,3
4,2013-07-01 00:04:51,0,1,7,2013,0,4


In [16]:
day_type_code={
    "A":"Normal Day",
    "B":"Holiday",
    "C":"Day before Holiday"
}
day_type_dim=df[['DAY_TYPE']].reset_index(drop=True)
day_type_dim['day_type_id']=day_type_dim.index
day_type_dim['day_type_name']=day_type_dim['DAY_TYPE'].map(day_type_code)

In [17]:
day_type_dim.head()

,DAY_TYPE,day_type_id,day_type_name
0,A,0,Normal Day
1,A,1,Normal Day
2,A,2,Normal Day
3,A,3,Normal Day
4,A,4,Normal Day


In [18]:
call_type_code={
    "A":"Central",
    "B":"Taxi driver or Stand",
    "C":"Others"
}
call_type_dim=df[['CALL_TYPE']].reset_index(drop=True)
call_type_dim['call_type_id']=call_type_dim.index
call_type_dim['call_type_name']=call_type_dim['CALL_TYPE'].map(call_type_code)

In [19]:
call_type_dim.head()

,CALL_TYPE,call_type_id,call_type_name
0,C,0,Others
1,B,1,Taxi driver or Stand
2,C,2,Others
3,C,3,Others
4,C,4,Others


In [20]:
fact_table = df.merge(datetime_dim, left_on='trip_id', right_on='datetime_id') \
             .merge(day_type_dim, left_on='trip_id', right_on='day_type_id') \
             .merge(call_type_dim, left_on='trip_id', right_on='call_type_id') \
             [['trip_id', 'datetime_id', 'day_type_id','call_type_id']]

In [21]:
fact_table.head()

,trip_id,datetime_id,day_type_id,call_type_id
0,0,0,0,0
1,1,1,1,1
2,2,2,2,2
3,3,3,3,3
4,4,4,4,4
